In [2]:
%load_ext sql
%sql duckdb:///../data/referral.db --alias referralsDB

c:\Users\MatthewLyn\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


There's a new jupysql version available (0.7.0), you're running 0.5.5. To upgrade: pip install jupysql --upgrade


In [3]:
%sql referralsDB

In [7]:
%sql CREATE OR REPLACE TABLE hcpcs as SELECT DISTINCT * FROM read_csv_auto('../data/reference/cpt_hcpcs.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
16991


In [3]:
%sql CREATE OR REPLACE TABLE icd10cm as SELECT DISTINCT * FROM read_csv('../data/reference/icd10cm.csv', header=True, columns={'code_value': 'VARCHAR', 'code_long_description': 'VARCHAR'})

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
95387


In [4]:
%%sql 
CREATE OR REPLACE TABLE cln_referrals as 

    SELECT DISTINCT * 
    FROM read_parquet('../data/clean_referrals/*.parquet')

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
55510


- No Auth Required ->	Appointment Scheduled or Member to Schedule or Unable to Contact
- Approved	       ->   Appointment Scheduled or Member to Schedule or Unable to Contact
- Denied	       ->   Open
- Withdrawn	       ->   Open

In [8]:
%sql SELECT DISTINCT * FROM hcpcs LIMIT 25

*  (referralsDB) duckdb:///../data/referral.db
Done.


code_short_description,code_value,code_category
RPR F/E/E/N/L/M 7.6-12.5 CM,12015,CPT
INTMD RPR N-HF/GENIT 2.5CM/<,12041,CPT
EXC SHOULDER TUM DEEP < 5 CM,23076,CPT
FUSION OF STOMACH AND BOWEL,43810,CPT
NEEDLE BIOPSY SPINAL CORD,62269,CPT
US EXAM SCROTUM,76870,CPT
INJECT R VENTR/ATRIAL ANGIO,93566,CPT
PALB2 MRNA SEQ ALYS,0137U,CPT
TRACT STAND FREESTAND CERV TRACT,E0850,HCPCS
ADD LW EXT ISCHIAL CONTAINMENT SCKT,L5649,HCPCS


In [10]:
%%sql 
CREATE OR REPLACE TABLE bi_referrals as

    SELECT DISTINCT *

    , UPPER("Last Name") || UPPER("First Name") as patient_name

    , UPPER("Last Name") || UPPER("First Name") || "DOB" as patient_id

    , patient_id || coalesce("Plan ID", 'NA') || coalesce("Referring Provider NPI", 'NA') || "Referral Date" || coalesce("Procedure", 'NA') || coalesce(Specialty, 'NA') as Referral_keyid

    , CASE
        WHEN "Approval Status" IN ('No HP Auth Required', 'HP Approved', 'Approved (comments required)', 'Complete/no Auth# needed')  
            AND "Visit Status" IN ('Appointment Scheduled', 'Member to Schedule', 'Unable to Contact', 'Denied', 'Withdrawn')
            THEN 'Completed'
        WHEN "Approval Status" IN ('Denied- Per Insurance Plan', 'Denied- Per Medical Director Review')
            AND "Visit Status" = 'Open'
            THEN 'Completed'
        ELSE "Referral Status" END as "Updated Status"

    , RIGHT('Procedure', 5) as Proc_code

    , FORMAT(
        '({}) {}-{}'
        , substring("Mobile Phone", 1, 3), substring("Mobile Phone", 3, 3), substring("Mobile Phone", 6, 4)
    ) as "Fmt Mobile Phone"

    , FORMAT(
        '({}) {}-{}'
        , substring("Home Phone", 1, 3), substring("Home Phone", 3, 3), substring("Home Phone", 6, 4)
    ) as "Fmt Home Phone"

    , icd.code_long_description AS "Diagnosis Description"

    FROM cln_referrals AS rf

    LEFT JOIN icd10cm AS icd
        ON trim(rf."Diagnosis") = trim(icd.code_value)

    WHERE "Visit Status" IS NOT NULL

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
55510


In [12]:
%sql SELECT DISTINCT * FROM bi_referrals LIMIT 25

*  (referralsDB) duckdb:///../data/referral.db
Done.


C0,Center,Clinic,Referring Provider,Referring Provider NPI,Referral Date,Last Name,First Name,Sex,DOB,pat Status,Language,Address,Mobile Phone,Home Phone,PCP,Referred to Specialist,Specialist NPI,Referral Status,Network,Specialty,Diagnosis,Procedure,Approval Status,Visit Status,Referral Type,Patient Type,Health Plan,Plan ID,LOB,Auth No,Last Update,Update_DT,file_source,code_value,code_long_description,patient_name,patient_id,Referral_keyid,Updated Status,Proc_code,Fmt Mobile Phone,Fmt Home Phone,Diagnosis Description
15,"Centrum Medical Holdings, LLC",LMMC - 7200 (1F),"Akbar Nikfarjam, MD",1649651274,04-03-23,CAMACHO,NORMA,F,12-21-48,Active,Spanish,"6720 NW 114 AVE APT 807DORAL, FL, 33178",7865217052,7865217052,"Nikfarjam, Akbar",None,None,Open,Out of network,Dermatology,R23.4,None,None,Open,External,Adult,Preferred Care Network (Medica),966904422,Medicare,None,"Nikfarjam, Akbar",Apr 3 2023 12:05PM,../data/raw_referrals/excels/FL REFERRALS 3_31-4_5.xlsx,R23.4,CHANGES IN SKIN TEXTURE,CAMACHONORMA,CAMACHONORMA12-21-48,CAMACHONORMA12-21-48966904422164965127404-03-23NADermatology,Open,edure,(786) 652-1705,(786) 652-1705,CHANGES IN SKIN TEXTURE
45,"Centrum Medical Holdings, LLC",CH - L Havana 12,"Alberto Orta-Iglesias, MD",1386167575,04-03-23,URDANETA mendoza,MAGALLY,F,11-08-58,Active,Spanish,"2415 NW 16TH STREET RDMIAMI, FL, 33125",7867973541,7867973541,Orta,None,None,Open,Out of network,Ophthalmology,H53.8,None,No HP Auth Required,Pending Appt,External,Adult,Oscar Insurance Company of Florida,OSC78634389-01,Exchange,None,"Cherlo, Esther",Apr 3 2023 11:25AM,../data/raw_referrals/excels/FL REFERRALS 3_31-4_5.xlsx,H53.8,OTHER VISUAL DISTURBANCES,URDANETA MENDOZAMAGALLY,URDANETA MENDOZAMAGALLY11-08-58,URDANETA MENDOZAMAGALLY11-08-58OSC78634389-01138616757504-03-23NAOphthalmology,Open,edure,(786) 679-7354,(786) 679-7354,OTHER VISUAL DISTURBANCES
55,"Centrum Medical Holdings, LLC",CH - L Havana 12,"Alberto Orta-Iglesias, MD",1386167575,03-31-23,CASTILLO CUETA,IRIS,F,04-09-65,Active,Spanish,"1452 SW 5 ST APT # 9MIAMI, FL, 33135",7866552242,7866552242,Orta,pending,None,Open,Out of network,Dentistry,Z87.891,OFFICE OUTPATIENT NEW 45 MINUTES - 99204,No HP Auth Required,Pending Appt,External,Adult,Simply (Medicaid) Med-Care,8904592054,Medicaid,N/A,"DePosada, Yorday",Apr 4 2023 1:39PM,../data/raw_referrals/excels/FL REFERRALS 3_31-4_5.xlsx,Z87.891,PERSONAL HISTORY OF NICOTINE DEPENDENCE,CASTILLO CUETAIRIS,CASTILLO CUETAIRIS04-09-65,CASTILLO CUETAIRIS04-09-658904592054138616757503-31-23OFFICE OUTPATIENT NEW 45 MINUTES - 99204Dentistry,Open,edure,(786) 665-5224,(786) 665-5224,PERSONAL HISTORY OF NICOTINE DEPENDENCE
255,"Centrum Medical Holdings, LLC",LMMC - HOMESTEAD,"Barbara Llanes, MD",1669812061,03-31-23,SOLORZANO,MARIA,F,09-12-87,Active,Spanish,"13420 SW 284 TERHOMESTEAD, FL, 33033",7866706271,7866706271,"Llanes, Barbara",None,None,Open,Out of network,Cardiology,I49.3,None,None,Open,External,Adult,Ambetter (Centrum Contract),U9697746002,Exchange,None,"Llanes, Barbara",Mar 31 2023 11:28AM,../data/raw_referrals/excels/FL REFERRALS 3_31-4_5.xlsx,I49.3,VENTRICULAR PREMATURE DEPOLARIZATION,SOLORZANOMARIA,SOLORZANOMARIA09-12-87,SOLORZANOMARIA09-12-87U9697746002166981206103-31-23NACardiology,Open,edure,(786) 667-0627,(786) 667-0627,VENTRICULAR PREMATURE DEPOLARIZATION
304,"Centrum Medical Holdings, LLC",CH - Margate,"Caridad Valdes, APRN",1609522788,04-03-23,LOPEZ PINO,INES,F,08-09-58,Active,Spanish,"3650 Inverrary Dr Apt 2PLauderhill, FL, 33319",3059009743,3059009743,"Valdes, Caridad",Appt by PT,None,Open,Out of network,Endocrinology,M85.80,OFFICE CONSULTATION NEW/ESTAB PATIENT 30 MIN - 99242,HP Auth Required,Pending Appt,External,Adult,Ambetter (Centrum Contract),U9657205801,Exchange,N/A,"Rengifo, Stephania",Apr 3 2023 8:45AM,../data/raw_referrals/excels/FL REFERRALS 3_31-4_5.xlsx,M85.80,OTH SPEC D/O BONE DENSITY STRUCTURE UNS SITE,LOPEZ PINOINES,LOPEZ PINOINES08-09-58,LOPEZ PINOINES08-09-58U9657205801160952278804-03-23OFFICE CONSULTATION NEW/ES

In [6]:
%%sql 
COPY (SELECT * FROM bi_referrals) TO '../data/report_data/2023-04-05.parquet' (FORMAT 'parquet'); 

*  (referralsDB) duckdb:///../data/referral.db


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

Done.


Count
55510
